#### Get ionbot features of the IDs

In [1]:
# import stuff...
import pandas as pd
import os
from IPython.display import clear_output

In [2]:
# indicate directory where all the data is present
# all other paths in the code will be changed accordingly
# using absolute path to the directory to avoid confusion...
working_folder = "C:/Users/Enrico/OneDrive - UGent/run-ionbot"
# working_folder = "D:/run-ionbot"
PXDs = [
    'PXD002057.v0.11.4',
    'PXD005833.v0.11.4',
    'PXD014258.v0.11.4',
    'PXD002057-closed',
    'PXD005833-closed',
    'PXD014258-closed'
]
SEARCHES = [
    'canon',
    'trembl',
    'openprot',
]
DATASETS = pd.MultiIndex.from_product([PXDs,SEARCHES])
DATASETS

# I organized the data like this:
# WorkingFolder/
#   PXD014258.v0.11.4/
#       PXD014258.v0.11.4-canon/
#       PXD014258.v0.11.4-trembl/
#       PXD014258.v0.11.4-openprot/
# etc...

MultiIndex([('PXD002057.v0.11.4',    'canon'),
            ('PXD002057.v0.11.4',   'trembl'),
            ('PXD002057.v0.11.4', 'openprot'),
            ('PXD005833.v0.11.4',    'canon'),
            ('PXD005833.v0.11.4',   'trembl'),
            ('PXD005833.v0.11.4', 'openprot'),
            ('PXD014258.v0.11.4',    'canon'),
            ('PXD014258.v0.11.4',   'trembl'),
            ('PXD014258.v0.11.4', 'openprot'),
            ( 'PXD002057-closed',    'canon'),
            ( 'PXD002057-closed',   'trembl'),
            ( 'PXD002057-closed', 'openprot'),
            ( 'PXD005833-closed',    'canon'),
            ( 'PXD005833-closed',   'trembl'),
            ( 'PXD005833-closed', 'openprot'),
            ( 'PXD014258-closed',    'canon'),
            ( 'PXD014258-closed',   'trembl'),
            ( 'PXD014258-closed', 'openprot')],
           )

In [3]:
# loop thought ionbot results
# for each search, merge ionbot.first (ids) and ionbot.features files, then concatenate them into one big df
for dataset_name,search in DATASETS: 
    clear_output(wait=True)
    PATH = os.path.join(working_folder,dataset_name,f'{dataset_name}-{search}') # edit this line so it fits your folder structure
    print(PATH)
    combo = []
    for i,_ in enumerate(os.scandir(PATH)):
        if os.path.isdir(_):
            print(_)
            features = pd.read_csv(os.path.join(_.path,'ionbot.features.csv'))
            peptides = pd.read_csv(os.path.join(_.path,'ionbot.first.csv'))
            peptides.spectrum_file = peptides.spectrum_file.apply(lambda x: x.split('.')[0])
            tmp = peptides[['ionbot_match_id','spectrum_title','scan','spectrum_file']].merge(features, on='ionbot_match_id')
            if len(features)<len(peptides) or len(tmp)!=len(peptides):
                print("Please double check:", len(features), len(peptides), len(tmp)) 
                # if len(tmp)!=len(peptides) something went wrong...
            combo.append(tmp)
    del features, peptides, tmp
    combo = pd.concat(combo, ignore_index=False)

    # in some mgf files the 'spectrum title' includes the file name, making the spectrum title unique.
    # when the file name is NOT included, spectrum titles are NOT unique, and this can mess up some analysis.
    combo.spectrum_title = combo.spectrum_file + ':' + combo.spectrum_title.apply(lambda x: x.split(':')[-1])
    
    combo.to_csv(os.path.join(PATH,f'{dataset_name}-{search}-combined-features.csv.gz'), index=False, compression='gzip')

print('\nDone!')

C:/Users/Enrico/OneDrive - UGent/run-ionbot\PXD014258-closed\PXD014258-closed-openprot
<DirEntry 'Sample-BT474-closed-open'>
<DirEntry 'Sample-MCF-closed-open'>
<DirEntry 'SampleHela-closed-open'>

Done!


----------